[View in Colaboratory](https://colab.research.google.com/github/utensil/julia-playground/blob/master/dl/hello_mx_gpu_true.ipynb)

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!apt install libnvrtc8.0
!pip install mxnet-cu80
import mxnet as mx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libnvrtc8.0
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 6,225 kB of archives.
After this operation, 28.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvrtc8.0 amd64 8.0.61-1 [6,225 kB]
Fetched 6,225 kB in 1s (5,420 kB/s)

78Selecting previously unselected package libnvrtc8.0:amd64.
(Reading database ... 86790 files and directories currently installed.)
Preparing to unpack .../libnvrtc8.0_8.0.61-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 16%] [#########.................................................] 8Unpacking libnvrtc8.0:amd64 (8.0.61-1) ...
7Progress: [ 33%] [###################.......................................] 87Progress: [ 50%] [#############################........

    100% |████████████████████████████████| 299.1MB 47kB/s 


From https://gluon.mxnet.io/chapter03_deep-neural-networks/mlp-gluon.html :

In [0]:
from __future__ import print_function
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon

data_ctx = mx.cpu()
model_ctx = mx.gpu(0)

In [0]:
batch_size = 64
num_inputs = 784
num_outputs = 10
num_examples = 60000
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)

In [0]:
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [0]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = gluon.nn.Dense(64)
            self.dense1 = gluon.nn.Dense(64)
            self.dense2 = gluon.nn.Dense(10)

    def forward(self, x):
        x = nd.relu(self.dense0(x))
        x = nd.relu(self.dense1(x))
        x = self.dense2(x)
        return x

In [0]:
net = MLP()
net.collect_params().initialize(mx.init.Normal(sigma=.01), ctx=model_ctx)

In [8]:
data = nd.ones((1,784))
net(data.as_in_context(model_ctx))


[[ 0.0010479  -0.00023263  0.00024665 -0.00137001 -0.00089217 -0.00043491
   0.00174529 -0.00114445  0.00024293 -0.0004818 ]]
<NDArray 1x10 @gpu(0)>

In [9]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = gluon.nn.Dense(64, activation="relu")
            self.dense1 = gluon.nn.Dense(64, activation="relu")
            self.dense2 = gluon.nn.Dense(10)

    def forward(self, x):
        x = self.dense0(x)
        print("Hidden Representation 1: %s" % x)
        x = self.dense1(x)
        print("Hidden Representation 2: %s" % x)
        x = self.dense2(x)
        print("Network output: %s" % x)
        return x

net = MLP()
net.collect_params().initialize(mx.init.Normal(sigma=.01), ctx=model_ctx)
net(data.as_in_context(model_ctx))

Hidden Representation 1: 
[[0.         0.         0.07143721 0.         0.         0.02748545
  0.33361885 0.2867228  0.         0.         0.5456277  0.21671814
  0.20799856 0.2859339  0.         0.34327483 0.         0.4614671
  0.02448964 0.         0.01388636 0.         0.25961956 0.
  0.15838279 0.         0.10904857 0.         0.         0.02812487
  0.         0.         0.3602645  0.         0.3662219  0.
  0.36324638 0.19642079 0.35552478 0.01087478 0.6817804  0.
  0.         0.02232164 0.02738917 0.         0.         0.
  0.05049316 0.37144935 0.16642064 0.         0.07810777 0.01600346
  0.         0.         0.         0.         0.         0.
  0.         0.         0.07678016 0.6952986 ]]
<NDArray 1x64 @gpu(0)>
Hidden Representation 2: 
[[0.00646666 0.         0.01819706 0.         0.         0.00655819
  0.         0.         0.         0.         0.         0.02194543
  0.         0.         0.00290144 0.         0.         0.00796402
  0.         0.         0.00674235


[[ 0.00144656  0.00042265 -0.00119047  0.00066365  0.00035064  0.00032936
  -0.00058919 -0.00066042 -0.00112773 -0.00013794]]
<NDArray 1x10 @gpu(0)>

In [0]:
num_hidden = 64
net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))
net.hybridize()

In [0]:
net.collect_params().initialize(mx.init.Normal(sigma=.1), ctx=model_ctx)

In [0]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .01})

In [0]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [15]:
!pip install mxboard

    100% |████████████████████████████████| 51kB 2.3MB/s 


In [0]:
from mxboard import SummaryWriter
sw = SummaryWriter(logdir='logs', flush_secs=5)

In [20]:
epochs = 10
smoothing_constant = .01

# collect parameter names for logging the gradients of parameters in each epoch
params = net.collect_params()
param_names = params.keys()
global_step = 0

for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
 
        sw.add_scalar(tag='cross_entropy', value=loss.mean().asscalar(), global_step=global_step)
        if i == 0:
            sw.add_image('minist_first_minibatch', data.reshape((batch_size, 1, 28, 28)), e)
        if e == 0:
            sw.add_graph(net)
        grads = [i.grad() for i in net.collect_params().values()]
        for i, name in enumerate(param_names):
            sw.add_histogram(tag=name, values=grads[i], global_step=e, bins=1000)

        global_step += 1
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss).asscalar()

        
    
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    sw.add_scalar(tag='accuracy_curves', value=('train_acc', train_accuracy), global_step=e)
    sw.add_scalar(tag='accuracy_curves', value=('valid_acc', test_accuracy), global_step=e);
    print ("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, cumulative_loss/num_examples, train_accuracy, test_accuracy)) 

Epoch 0. Loss: 0.45416317160924274, Train_acc 0.886166666667, Test_acc 0.8904
Epoch 1. Loss: 0.367962058989207, Train_acc 0.90005, Test_acc 0.9012
Epoch 2. Loss: 0.3281178052981695, Train_acc 0.909833333333, Test_acc 0.9114
Epoch 3. Loss: 0.30091046256224313, Train_acc 0.916866666667, Test_acc 0.9165
Epoch 4. Loss: 0.27957490228017173, Train_acc 0.92285, Test_acc 0.922
Epoch 5. Loss: 0.2620397605220477, Train_acc 0.92685, Test_acc 0.925
Epoch 6. Loss: 0.24704794867038726, Train_acc 0.9311, Test_acc 0.9298
Epoch 7. Loss: 0.23364521889686585, Train_acc 0.934766666667, Test_acc 0.9327
Epoch 8. Loss: 0.22148989040851594, Train_acc 0.938133333333, Test_acc 0.9346
Epoch 9. Loss: 0.21068138181765875, Train_acc 0.940883333333, Test_acc 0.9377


In [22]:
!ls logs

accuracy_curves  events.out.tfevents.1532778751.2f6afaea2f18


In [0]:
!zip -r logs.zip logs/

  adding: logs/ (stored 0%)
  adding: logs/events.out.tfevents.1532778751.2f6afaea2f18 (deflated 58%)
  adding: logs/accuracy_curves/ (stored 0%)
  adding: logs/accuracy_curves/valid_acc/ (stored 0%)
  adding: logs/accuracy_curves/valid_acc/events.out.tfevents.1532778960.2f6afaea2f18 (deflated 54%)
  adding: logs/accuracy_curves/train_acc/ (stored 0%)
  adding: logs/accuracy_curves/train_acc/events.out.tfevents.1532778960.2f6afaea2f18 (deflated 55%)


In [0]:
from google.colab import files
files.download('logs.zip')

In [29]:
!ls -lhta logs.zip

-rw-r--r-- 1 root root 407M Jul 28 12:52 logs.zip


In [36]:
!pwd

/content


In [0]:
LOG_DIR = '/content/logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 127.0.0.1 --port 6006 & || echo failed'
    .format(LOG_DIR)
)

In [33]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-07-28 12:59:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.39.76, 52.22.213.157, 52.23.126.223, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.39.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  19.9MB/s    in 0.3s    

2018-07-28 12:59:50 (19.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [97]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://31c5710b.ngrok.io


In [98]:
!python /usr/local/lib/python2.7/dist-packages/tensorboard/main.py  --logdir /content/logs --host 0.0.0.0 --port 6006 2>&1|grep -v Reloader

TensorBoard 1.9.0 at http://0.0.0.0:6006 (Press CTRL+C to quit)
^C
